In [1]:
import pandas as pd
import numpy as np

#读取文件
mobility_raw = pd.read_csv('sojournTime.csv',index_col=0)
delay_raw    = pd.read_csv('delay.csv',index_col=0)
load_raw     = pd.read_csv('load.csv',index_col=0)
compute_raw  = pd.read_csv('memory.csv',index_col=0)

# 得到cloudlet数量
cloudlet_numbers = mobility_raw.shape[1]

# 对delay和load进行同向化
delay_txh = delay_raw.apply(lambda x: (np.max(x) - x)) 
load_txh = load_raw.apply(lambda x: (np.max(x) - x)) 

# 合并成一个DataFrame
data = pd.concat([mobility_raw,delay_txh,load_txh,compute_raw],axis=1)

# 对所有数据进行归一化
data_nor = data.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
data_nor


,sojournTime1,sojournTime2,sojournTime3,sojournTime4,sojournTime5,sojournTime6,sojournTime7,sojournTime8,sojournTime9,sojournTime10,...,memory11,memory12,memory13,memory14,memory15,memory16,memory17,memory18,memory19,memory20
T1,0.379310,0.216216,0.142857,0.238095,0.973684,0.56,0.461538,0.421053,0.194444,0.52,...,0.000000,0.157895,0.083333,0.944444,0.032258,0.294118,0.000000,0.03125,0.074074,0.972222
T2,0.689655,0.297297,0.571429,0.666667,0.210526,0.00,0.230769,0.789474,0.027778,0.32,...,0.617647,0.289474,0.666667,0.666667,0.129032,0.088235,0.179487,1.00000,0.777778,0.555556
T3,0.758621,0.297297,0.380952,0.142857,0.263158,0.88,0.000000,0.526316,0.555556,1.00,...,1.000000,0.394737,0.958333,0.833333,0.258065,0.000000,0.871795,0.78125,1.000000,0.000000
T4,0.448276,0.189189,0.571429,0.000000,0.000000,0.20,0.769231,0.210526,0.527778,0.52,...,0.117647,0.947368,1.000000,1.000000,0.193548,0.176471,0.666667,0.28125,0.222222,0.333333
T5,0.448276,0.324324,0.000000,0.380952,1.000000,0.44,0.846154,0.473684,1.000000,0.52,...,0.882353,0.368421,0.125000,1.000000,0.967742,1.000000,0.666667,0.37500,0.000000,0.888889
T6,0.000000,0.513514,0.380952,0.142857,0.184211,0.04,0.128205,1.000000,0.861111,0.76,...,0.352941,1.000000,0.666667,0.666667,0.322581,0.382353,0.743590,0.18750,0.333333,0.722222
T7,0.517241,0.000000,1.000000,0.476190,0.500000,0.40,1.000000,0.421053,0.694444,0.20,...,0.970588,0.473684,0.791667,1.000000,1.000000,0.676471,0.743590,0.00000,0.777778,1.000000
T8,0.379310,1.000000,0.809524,0.571429,0.157895,0.28,0.230769,0.842105,0.416667,0.24,...,0.823529,0.184211,0.000000,0.000000,0.483871,0.117647,1.000000,0.81250,0.666667,0.222222
T9,1.000000,0.864865,0.571429,1.000000,0.210526,0.84,0.461538,0.789474,0.000000,0.00,...,0.500000,0.000000,0.708333,0.055556,0.000000,0.264706,0.846154,0.71875,0.666667,0.111111
T10,0.931034,0.513514,0.809524,0.095238,0.710526,1.00,0.538462,0.000000,0.388889,0.60,...,0.911765,0.473684,0.916667,0.333333,0.580645,0.588235,0.846154,0.00000,0.444444,0.111111


In [2]:
# 云模型处理
def cloud_model(df):
    Ex = df.mean()
    En = np.sqrt(np.pi/2)  * np.mean( np.abs(df-Ex) )
    He = np.sqrt(np.abs(df.mean() * df.mean() - df.std() * df.std()))
    return pd.concat([Ex,En,He],axis=1)
data_cloudModel = cloud_model(data_nor)
data_cloudModel

,0,1,2
sojournTime1,0.555172,0.290423,0.469156
sojournTime2,0.421622,0.302150,0.286288
sojournTime3,0.523810,0.298408,0.423247
sojournTime4,0.371429,0.310344,0.203249
sojournTime5,0.421053,0.375994,0.223297
...,...,...,...
memory16,0.358824,0.303744,0.178625
memory17,0.656410,0.284085,0.574232
memory18,0.418750,0.410460,0.179106
memory19,0.496296,0.352785,0.367818


In [3]:
# 中智聚合

# 中智数
truth = data_cloudModel[0]
indeterminacy = data_cloudModel[1]
falsity = data_cloudModel[2]

# 权重
weight = [0.25,0.25,0.25,0.25]

# 聚合之后的中智数
aggre_truth = []
aggre_indeterminacy = []
aggre_falsity = []

for i in range(cloudlet_numbers):
    tempT = 1
    tempI = 1
    tempF = 1
    # 四个属性
    for j in range(4):
        tempT *= (1 - truth[i+10*j]) ** weight[j]
        tempI *= indeterminacy[i+10*j] ** weight[j]
        tempF *= falsity[i+10*j] ** weight[j]
    aggre_truth.append((1-tempT))
    aggre_indeterminacy.append(tempI)
    aggre_falsity.append(tempF)

# 添加索引
index = pd.Series(np.arange(1,cloudlet_numbers+1))
index = index.astype(str)
index = 'cloudlet'+index
neutrosophic = pd.DataFrame(np.vstack((aggre_truth,aggre_indeterminacy,aggre_falsity)).T,index=index)
neutrosophic


,0,1,2
cloudlet1,0.526447,0.314963,0.371707
cloudlet2,0.509908,0.301745,0.381306
cloudlet3,0.470842,0.341676,0.313300
cloudlet4,0.507993,0.336930,0.322545
cloudlet5,0.455263,0.320972,0.301983
cloudlet6,0.464035,0.334095,0.218249
cloudlet7,0.536896,0.299015,0.421487
cloudlet8,0.484955,0.327636,0.315251
cloudlet9,0.485953,0.373954,0.325153
cloudlet10,0.454860,0.315618,0.312730


In [4]:
# 得分函数
score = neutrosophic[0]+1-neutrosophic[1]+1-neutrosophic[2]
print(score)
print('最佳cloudlet为：',score.idxmax())

with open("optimal_cloudlet.txt","w") as f:
        f.write(score.idxmax())  #这句话自带文件关闭功能，不需要再写f.close()


cloudlet1     1.839776
cloudlet2     1.826857
cloudlet3     1.815865
cloudlet4     1.848517
cloudlet5     1.832308
cloudlet6     1.911691
cloudlet7     1.816393
cloudlet8     1.842069
cloudlet9     1.786846
cloudlet10    1.826512
cloudlet11    1.858572
cloudlet12    1.811123
cloudlet13    1.807544
cloudlet14    1.808509
cloudlet15    1.828950
cloudlet16    1.948814
cloudlet17    1.802187
cloudlet18    1.862366
cloudlet19    1.798278
cloudlet20    1.831197
dtype: float64
最佳cloudlet为： cloudlet16
